# Statistically, what would be the best Pokemon team Ash could assemble?

Members: Sathvik Inteti, Arnav Mittal, Daneil Nguyen

## Introduction

Pokemon is a famous video game series that has beloved fans both old and young. It revolves around "trainers" catching these creatures called Pokemon and using them in battles against others. It has become an international sensation and has spawned a multitude of other forms of entertainment such as a trading card game, movies, and an anime show. Most notably, many fans know of 10 year old Ash Ketchum, the protagonist of the Pokemon anime. The anime follows Ash and his friends on his journey to become a Pokemon Champion, the best Pokemon trainer of them all.

The Pokemon anime has been on air for more than 20 years and Ash has not aged one single bit (because of cartoon logic). Additionally, he has yet to become a Pokemon Champion despite the amount of time he has trained and the number of Pokemon he has encountered, until recently. In 2019, after 22 years after the initial release of the show, Ash's dream came into fruition as he has conquered the Alola league and was crowned as a Pokemon Champion ([source](https://www.pokemonfanclub.net/did-ash-become-a-pokemon-master/#:~:text=Pokemons%20Ash%20Finally%20Becomes%20A,master%20for%20the%20first%20time.)). 

Here, a question arises that we would like to explore. Ash has had 22 years of experience and has accumulated a grand roster over this time. What is the best Pokemon team that Ash could assemble in terms of strength?

In our case, "strength" is determined by total stats of a Pokemon and how a specific Pokemon's type fairs with others (i.e. what types are they weak to and strong against). There are other factors that come into play such as abilities, the moves that Pokemon can use, etc. but that adds an extra level of complexity that the datasets do not currently support. 

To test how "strong" our team is, we will face it against Ash's original rival in the Pokemon anime and video game: Gary Oak. We will be using Gary's lineup in which he chose Squirtle as his starter Pokemon as for the most part many players of the Pokemon game chose to obtain Charmander (the rival will always pick the Pokemon that has a type advantage against yours).

We will be utilizing various data sets that contain information about the stats of various Pokemon along with combat data in which we will feed to a learning model to predict the winner of this grand battle between Pokemon champions. 






Link to datasets used:

Pokemon Stats Data: https://www.kaggle.com/datasets/mariotormo/complete-pokemon-dataset-updated-090420?select=pokedex_%28Update.04.20%29.csv

Pokemon Combat Data: https://www.kaggle.com/datasets/terminus7/pokemon-challenge?select=tests.csv

Ash Pokemon Data: https://www.epicdope.com/all-of-ashs-pokemon-in-the-anime/

Gary Pokemon Data: https://pokemon.fandom.com/wiki/Blue%27s_Pok%C3%A9mon_Teams_(Red/Green/Blue)#Charmander


## Data Wrangling

To get started, we will need to clean up the datasets that we will be using by extracting only the relevant columns along with rows of Pokemon. For Ash and Gary, we will store information about the Pokemon that they have and with Ash we will do further scraping to get the best lineup he can use against Gary. 

In [153]:
#import all necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

In [154]:
#import pokemon data from csv and filter by necessary columns
all_pokemon = pd.read_csv('data/pokedex_(Update.04.20).csv')
all_pokemon = all_pokemon[['pokedex_number', 'name', 'type_1', 'type_2', 'total_points', 'against_normal', 'against_fire', 'against_water', 'against_electric', 'against_grass', 'against_ice', 'against_fight', 'against_poison', 'against_ground', 'against_flying', 'against_psychic', 'against_bug', 'against_rock', 'against_ghost', 'against_dragon', 'against_dark', 'against_steel', 'against_fairy']]
all_pokemon = all_pokemon.set_axis(['pokedex_number', 'name', 'type_1', 'type_2', 'total_points', 'Normal', 'Fire', 'Water', 'Electric', 'Grass', 'Ice', 'Fight', 'Poison', 'Ground', 'Flying', 'Psychic', 'Bug', 'Rock', 'Ghost', 'Dragon', 'Dark', 'Steel', 'Fairy'], axis=1, inplace=False)
all_pokemon.head()

,pokedex_number,name,type_1,type_2,total_points,Normal,Fire,Water,Electric,Grass,...,Ground,Flying,Psychic,Bug,Rock,Ghost,Dragon,Dark,Steel,Fairy
0,1,Bulbasaur,Grass,Poison,318.0,1.0,2.0,0.5,0.5,0.25,...,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,0.5
1,2,Ivysaur,Grass,Poison,405.0,1.0,2.0,0.5,0.5,0.25,...,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,0.5
2,3,Venusaur,Grass,Poison,525.0,1.0,2.0,0.5,0.5,0.25,...,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,0.5
3,3,Mega Venusaur,Grass,Poison,625.0,1.0,1.0,0.5,0.5,0.25,...,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,0.5
4,4,Charmander,Fire,NaN,309.0,1.0,0.5,2.0,1.0,0.50,...,2.0,1.0,1.0,0.5,2.0,1.0,1.0,1.0,0.5,0.5


In [155]:
#get Gary's list of Pokemon and retrieve data on each
gary_dex = pd.read_csv('data/Gary_pokemon.csv')
gary_pokemon = pd.DataFrame()
for i, r in gary_dex.iterrows():
    gary_pokemon = pd.concat([gary_pokemon, all_pokemon[all_pokemon['name'] == r['Name']]])
gary_pokemon

,pokedex_number,name,type_1,type_2,total_points,Normal,Fire,Water,Electric,Grass,...,Ground,Flying,Psychic,Bug,Rock,Ghost,Dragon,Dark,Steel,Fairy
22,18,Pidgeot,Normal,Flying,479.0,1.0,1.0,1.0,2.0,0.5,...,0.0,1.0,1.0,0.5,2.0,0.0,1.0,1.0,1.0,1.0
83,65,Alakazam,Psychic,NaN,500.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.5,2.0,1.0,2.0,1.0,2.0,1.0,1.0
144,112,Rhydon,Ground,Rock,485.0,0.5,0.5,4.0,0.0,4.0,...,2.0,0.5,1.0,1.0,0.5,1.0,1.0,1.0,2.0,1.0
77,59,Arcanine,Fire,NaN,555.0,1.0,0.5,2.0,1.0,0.5,...,2.0,1.0,1.0,0.5,2.0,1.0,1.0,1.0,0.5,0.5
132,103,Exeggutor,Grass,Psychic,530.0,1.0,2.0,0.5,0.5,0.5,...,0.5,2.0,0.5,4.0,1.0,2.0,1.0,2.0,1.0,1.0
11,9,Blastoise,Water,NaN,530.0,1.0,0.5,0.5,2.0,2.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.5,1.0


In [156]:
#get Ash's list of Pokemon and retrieve data on each
ash_dex = pd.read_csv('data/Ash_pokemon.csv')
ash_pokemon = pd.DataFrame()
for i, r in ash_dex.iterrows():
    ash_pokemon = pd.concat([ash_pokemon, all_pokemon[all_pokemon['pokedex_number'] == r['Pokedex Number']]])
ash_pokemon = ash_pokemon.drop_duplicates('pokedex_number')
ash_pokemon

,pokedex_number,name,type_1,type_2,total_points,Normal,Fire,Water,Electric,Grass,...,Ground,Flying,Psychic,Bug,Rock,Ghost,Dragon,Dark,Steel,Fairy
193,153,Bayleef,Grass,NaN,405.0,1.0,2.00,0.5,0.5,0.50,...,0.50,2.0,1.0,2.00,1.00,1.0,1.0,1.00,1.0,1.0
616,525,Boldore,Rock,NaN,390.0,0.5,0.50,2.0,1.0,2.00,...,2.00,0.5,1.0,1.00,1.00,1.0,1.0,1.00,2.0,1.0
497,418,Buizel,Water,NaN,330.0,1.0,0.50,0.5,2.0,2.00,...,1.00,1.0,1.0,1.00,1.00,1.0,1.0,1.00,0.5,1.0
0,1,Bulbasaur,Grass,Poison,318.0,1.0,2.00,0.5,0.5,0.25,...,1.00,2.0,2.0,1.00,1.00,1.0,1.0,1.00,1.0,0.5
15,12,Butterfree,Bug,Flying,395.0,1.0,2.00,1.0,2.0,0.25,...,0.00,2.0,1.0,0.50,4.00,1.0,1.0,1.00,1.0,1.0
6,6,Charizard,Fire,Flying,534.0,1.0,0.50,2.0,2.0,0.25,...,0.00,1.0,1.0,0.25,4.00,1.0,1.0,1.00,0.5,0.5
402,341,Corphish,Water,NaN,308.0,1.0,0.50,0.5,2.0,2.00,...,1.00,1.0,1.0,1.00,1.00,1.0,1.0,1.00,0.5,1.0
278,232,Donphan,Ground,NaN,500.0,1.0,1.00,2.0,0.0,2.00,...,1.00,1.0,1.0,1.00,0.50,1.0,1.0,1.00,1.0,1.0
1016,882,Dracovish,Water,Dragon,505.0,1.0,0.25,0.0,1.0,1.00,...,1.00,1.0,1.0,1.00,1.00,1.0,2.0,1.00,0.5,2.0
187,149,Dragonite,Dragon,Flying,600.0,1.0,0.50,0.5,1.0,0.25,...,0.00,1.0,1.0,0.50,2.00,1.0,2.0,1.00,1.0,2.0


We will also need to see what the average stat distribution of Gary's Pokemon are and by doing so we can further filter down which of Ash's Pokemon can be utilized. 

In [157]:
gary_avg_total_stat = gary_pokemon['total_points'].mean()
print(gary_avg_total_stat)

513.1666666666666


Now that we have the average total stat points of Gary's Pokemon, we would ideally want to have all Pokemons that would be placed on Ash's team to have an equal or higher total stat point. 

In [158]:
ash_stat = ash_pokemon[ash_pokemon['total_points'] >= gary_avg_total_stat].sort_values(['total_points'], ascending=False)
ash_stat

,pokedex_number,name,type_1,type_2,total_points,Normal,Fire,Water,Electric,Grass,...,Ground,Flying,Psychic,Bug,Rock,Ghost,Dragon,Dark,Steel,Fairy
187,149,Dragonite,Dragon,Flying,600.0,1.0,0.5,0.5,1.0,0.25,...,0.0,1.0,1.0,0.50,2.00,1.0,2.0,1.0,1.0,2.0
815,706,Goodra,Dragon,NaN,600.0,1.0,0.5,0.5,0.5,0.00,...,1.0,1.0,1.0,1.00,1.00,1.0,2.0,1.0,1.0,2.0
939,809,Melmetal,Steel,NaN,600.0,0.5,2.0,1.0,1.0,0.50,...,2.0,0.5,0.5,0.50,0.50,1.0,0.5,1.0,0.5,0.5
934,804,Naganadel,Poison,Dragon,540.0,1.0,0.5,0.5,0.5,0.25,...,2.0,1.0,2.0,0.50,1.00,1.0,2.0,1.0,1.0,1.0
181,143,Snorlax,Normal,NaN,540.0,1.0,1.0,1.0,1.0,1.00,...,1.0,1.0,1.0,1.00,1.00,0.0,1.0,1.0,1.0,1.0
167,131,Lapras,Water,Ice,535.0,1.0,1.0,0.0,2.0,2.00,...,1.0,1.0,1.0,1.00,2.00,1.0,1.0,1.0,1.0,1.0
830,715,Noivern,Flying,Dragon,535.0,1.0,0.5,0.5,1.0,0.25,...,0.0,1.0,1.0,0.50,2.00,1.0,2.0,1.0,1.0,2.0
6,6,Charizard,Fire,Flying,534.0,1.0,0.5,2.0,2.0,0.25,...,0.0,1.0,1.0,0.25,4.00,1.0,1.0,1.0,0.5,0.5
469,392,Infernape,Fire,Fighting,534.0,1.0,0.5,2.0,1.0,0.50,...,2.0,2.0,2.0,0.25,1.00,1.0,1.0,0.5,0.5,1.0
764,658,Greninja,Water,Dark,530.0,1.0,0.5,0.5,2.0,2.00,...,1.0,1.0,0.0,2.00,1.00,0.5,1.0,0.5,0.5,2.0


Now that we have a full list of both Ash's and Gary's Pokemons and their respective data, we can now filter down Ash's Pokemon by seeing which Pokemon have a type advantage against Gary's Pokemon. First, we need to see what Gary's Pokemon are weak to. 

In [159]:
# for each of Gary's pokemon, get a list of all the types that are 2 or 4x weak
# compile together and drop duplicates
# get number of pokemon weak to each type by 2x or 4x

#clean this up

#get counter
np_gary = gary_pokemon.to_numpy()
weakness_2 = {}
weakness_4 = {}
weakness_2_count = {}
weakness_4_count = {}

for i in gary_pokemon['pokedex_number']:
    weakness_2[i] = []
    weakness_4[i] = []

cols = gary_pokemon.columns.values.tolist()
for i in np_gary:
    for j in range (5, len(i)):
        if (i[j] == 2):
            weakness_2[i[0]].append(cols[j])
            if cols[j] in weakness_2_count.keys():
                cur_count = weakness_2_count[cols[j]] + 1
                weakness_2_count[cols[j]] = cur_count
            else: 
                weakness_2_count[cols[j]] = 1
        elif (i[j] == 4):
            weakness_4[i[0]].append(cols[j])
            if cols[j] in weakness_4_count.keys():
                cur_count = weakness_4_count[cols[j]] + 1
                weakness_4_count[cols[j]] = cur_count
            else: 
                weakness_4_count[cols[j]] = 4

type_weakness_4 = []
for idx, i in enumerate(weakness_4):
    type_weakness_4 = type_weakness_4 + weakness_4[i]
            
type_weakness_4 = list(set(type_weakness_4))

type_weakness_2 = []
for idx, i in enumerate(weakness_2):
    type_weakness_2 = type_weakness_2 + weakness_2[i]
            
type_weakness_2 = list(set(type_weakness_2))

print(type_weakness_4)
print(type_weakness_2)
print()
print(weakness_4_count)
print(weakness_2_count)


['Water', 'Bug', 'Grass']
['Electric', 'Ice', 'Poison', 'Ground', 'Steel', 'Ghost', 'Fight', 'Grass', 'Flying', 'Fire', 'Rock', 'Dark', 'Water', 'Bug']

{'Water': 4, 'Grass': 4, 'Bug': 4}
{'Electric': 2, 'Ice': 3, 'Rock': 2, 'Bug': 1, 'Ghost': 2, 'Dark': 2, 'Fight': 1, 'Ground': 2, 'Steel': 1, 'Water': 1, 'Fire': 1, 'Poison': 1, 'Flying': 1, 'Grass': 1}


In [165]:
types = type_weakness_4
for idx, i in enumerate(weakness_2_count):
    if weakness_2_count[i] >= 2:
        types.append(i)

print(types)

ash_stat[ash_stat['type_1'].isin(types) | ash_stat['type_2'].isin(types) ]



['Water', 'Bug', 'Grass', 'Electric', 'Ice', 'Rock', 'Ghost', 'Dark', 'Ground', 'Electric', 'Ice', 'Rock', 'Ghost', 'Dark', 'Ground', 'Electric', 'Ice', 'Rock', 'Ghost', 'Dark', 'Ground']


,pokedex_number,name,type_1,type_2,total_points,Normal,Fire,Water,Electric,Grass,...,Ground,Flying,Psychic,Bug,Rock,Ghost,Dragon,Dark,Steel,Fairy
167,131,Lapras,Water,Ice,535.0,1.0,1.0,0.0,2.0,2.0,...,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0
764,658,Greninja,Water,Dark,530.0,1.0,0.5,0.5,2.0,2.0,...,1.0,1.0,0.0,2.0,1.0,0.5,1.0,0.5,0.5,2.0
846,727,Incineroar,Fire,Dark,530.0,1.0,0.5,2.0,1.0,0.5,...,2.0,1.0,0.0,1.0,2.0,0.5,1.0,0.5,0.5,1.0
301,254,Sceptile,Grass,NaN,530.0,1.0,2.0,0.5,0.5,0.5,...,0.5,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0
466,389,Torterra,Grass,Ground,525.0,1.0,2.0,1.0,0.0,1.0,...,0.5,2.0,1.0,2.0,0.5,1.0,1.0,1.0,1.0,1.0
646,553,Krookodile,Ground,Dark,519.0,1.0,1.0,2.0,0.0,2.0,...,1.0,1.0,0.0,2.0,0.5,0.5,1.0,0.5,1.0,2.0


Now that we have a list of Gary's weaknesses, we will also need to see which of the Ash's current list of Pokemon 

## Data Exploration and Analysis

In [161]:
#box plot of final teams of gary's and ash based on stats 

In [162]:
#include stats vs typing scatter plot?
#specify list of types that a team is specifically weak to or not strong against?
#mean stats and type?

combats_df = pd.read_csv("data/combats.csv")
combats_df.head()

,First_pokemon,Second_pokemon,Winner
0,266,298,298
1,702,701,701
2,191,668,668
3,237,683,683
4,151,231,151


In [163]:
#split into training and testing model





## Conclusion

Pokemon has grown to be a complex game with many battle mechanics that we were not able to show fully in the above data analysis. However, we can clearly see the importance of stats and typing when it comes to gauging the best team suited for a particular battle. 